In [1]:
import numpy as np
import pyvisa
import time
import datetime
import os

In [2]:
def store(_out, _filename):

    with open(_filename, "a") as file:
        file.write(_out)

In [74]:
def main():

    try:
        rm = pyvisa.ResourceManager()
        print(' -> available ressources:', rm.list_resources())

        # connect via ETH
        inst = rm.open_resource('TCPIP0::10.5.5.5::inst0::INSTR')

        # connect via USB
        # inst = rm.open_resource('USB::0x0AAD::0x0119::022019943::INSTR')

        inst.read_termination = '\n'

        inst.write_termination = '\n'

        # Query the Identification string
        print(' -> connected to: ', inst.query('*IDN?'))

        # Clear instrument io buffers and status
        inst.clear()

        inst.write('GTR')

        inst.write('SYSTem:DISPlay:UPDate ON')

        # inst.write('*RST') # Reset device
        # inst.write('INIT:CONT OFF')  # Switch OFF the continuous sweep

    except:
        print(f" -> not connected")

    tmax_sec = 3*86400

    # get starttime as utc
    starttime = datetime.datetime.now()

    elapsed_time = 0

    # specify filename
    path = "/home/brotzer/Downloads/"
    filename = path+f"romy_fsr_{starttime.strftime('%Y%m%dT%H%M%S')}.csv"

    # create file with header if not existent
    if not os.path.isfile(filename):
        f = open(filename, "w")
        f.write("datetime,x_hz,y_dbm\n")
        f.close()

    while elapsed_time < tmax_sec:

        # get data
        _t = datetime.datetime.utcnow()
        try:
            _x = inst.query_ascii_values('CALC:MARK:FUNC:FPE:X?')
            _y = inst.query_ascii_values('CALC:MARK:FUNC:FPE:Y?')
        except:
            _x, _y = np.nan, np.nan

        # compute elapsed time since start
        _delta = datetime.datetime.now() - starttime
        elapsed_time = _delta.seconds

        # form output string
        out = f"{_t},{_x},{_y}\n"

        # write output
        store(out, filename)

        # Wait for 1 second
        time.sleep(1)

if __name__ == "__main__":
    main()


 -> not connected


UnboundLocalError: local variable 'inst' referenced before assignment

In [3]:
import pandas as pd
import obspy as obs
import matplotlib.pyplot as plt


In [18]:
def __smooth(y, npts, win="hanning"):

    from numpy import ones, convolve, hanning, nan

    if win == "hanning":
        win = hanning(npts)
    elif win == "boxcar":
        win = ones(npts)

    y_smooth = convolve(y, win/sum(win), mode='same')

    y_smooth[:npts//2] = nan
    y_smooth[-npts//2:] = nan
    return y_smooth

In [19]:
df0 = pd.read_csv("~/Downloads/romy_fsr_20240821T140958.csv")

starttime = obs.UTCDateTime(df0.datetime.iloc[0])

df0['time_sec'] = np.array([obs.UTCDateTime(_dt) - starttime for _dt in df0.datetime])

In [20]:
# df = df0[abs(df0.y_dbm) < 95]

In [21]:
df = df0.copy()

df['x_mod'] = np.where(abs(df.y_dbm) < 110, df.x_hz, np.nan)

df['x_mod_rel'] = df.x_mod - np.nanmean(df.x_mod[:100])


fsr = 8.92e6
n = 12
L = 11.2

df['disp_um'] = df.x_mod_rel*n*3*L/fsr*1e6


In [33]:
def __makeplot():

    import matplotlib.pyplot as plt

    Nrow, Ncol = 2, 1

    xscale = 1

    font = 12

    fig, ax = plt.subplots(Nrow, Ncol, figsize=(12, 5), sharex=True)

    plt.subplots_adjust(hspace=0.1)

    ax[0].scatter(df['time_sec']/3600, df.x_mod_rel*xscale, s=3)
    ax[0].plot(df['time_sec']/3600, __smooth(df.x_mod_rel*xscale, 300), "k")
    ax[0].ticklabel_format(useOffset=False)

    ax[0].set_ylim(-20, 10)

    ax[1].plot(df['time_sec']/3600, df.y_dbm)

    ax[0].grid(which="both", ls="--", color="grey", alpha=0.5, zorder=0)
    ax[1].grid(which="both", ls="--", color="grey", alpha=0.5, zorder=0)

    plt.show();
    return fig

fig = __makeplot();


In [31]:
# %matplotlib tk